In [1]:
import os
from PIL import Image as Im
import numpy as np
import pickle
from CCCode.imaging_process import Check, WavefrontPropagate, tie_algorithm
from CCCode.cc_math import img_scale, pearson_correlation_coefficient
from scipy import io

In [2]:
data_dir = "C:\\users\\chenchao\\.keras\\datasets\\HAM10000_images"
save_dir = "data\\datasets\\"

In [3]:
def compute_mse(img1, img2):
    return np.mean((img1-img2)**2)

In [4]:
def import_one_pic(fpath, crop_box, scaled_size):
    img = Im.open(fpath)
    img = img.crop(crop_box).convert("L")
    img.thumbnail((scaled_size, scaled_size))
    img_arr = np.asarray(img)
    return img_arr

In [5]:
def import_img(data_dir, save_dir, img_num, img_size, save_file=False):
    # import images from specific directory
    file_list = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    crop_box = (100, 25, 500, 425)
    img_array_datasets = np.zeros((img_num, img_size, img_size))
    for fpath, n in zip(file_list[:img_num], range(img_num)):
        print(n)
        img_arr = import_one_pic(fpath, crop_box, img_size)
        img_array_datasets[n, :, :] = img_arr
    return img_array_datasets

In [6]:
def propagating_process(img_arr, pixel_size, wave_length, delta, crop_size):
    # for every image in imported array, generating corresponding multi-focus images
    img_num = img_arr.shape[0] // 2
    multi_focus_arr = np.zeros((img_num, crop_size, crop_size, 4))
    for i in range(img_num):
        amp_img = img_arr[i]
        pha_img = img_arr[img_num+i]
        # the values of amplitude and phase will influence the recovering coef
        # with minier difference of amplitude value and larger difference of phase value
        # the coef of phase recovering will be better
        amp_img = img_scale(amp_img, img_min=1.0, img_max=1.08)
        pha_img = img_scale(pha_img, img_min=0.0, img_max=1.5)
        wf = WavefrontPropagate(crop_size, pixel_size, wave_length)
        wf.get_obj_wavefront(amp_img, pha_img)
        multi_focus_img = wf.directly_propagate(delta, crop_size)
        multi_focus_arr[i, :, :, 0] = multi_focus_img[0]
        multi_focus_arr[i, :, :, 1] = multi_focus_img[1]
        multi_focus_arr[i, :, :, 2] = multi_focus_img[2]
        multi_focus_arr[i, :, :, 3] = pha_img
        if i % 10 == 0:
            print(i)
    return multi_focus_arr

In [7]:
def single_evaluate(imgs, pixel_size, wave_length, delta, epsilon, crop_size, did_plot=False,
                    did_evaluate_mse=False, save_pxy=False):
    ground_truth = imgs[3]
    pxy = tie_algorithm(imgs[0], imgs[1], imgs[2],crop_size,
                        delta, pixel_size, wave_length, epsilon)
    diff = ground_truth - np.min(ground_truth) - (pxy - np.min(pxy))
    coef_value = pearson_correlation_coefficient(ground_truth, pxy)
    if did_evaluate_mse:
        mse_value = compute_mse(ground_truth, pxy)
    
    # visuallize
    if did_plot:
        ck = Check()
        ck.multi_img(False, ground_truth=ground_truth, py_recover=pxy, difference=diff)
    print("Coef: ", coef_value)
    if did_evaluate_mse:
        print("MSE: ", mse_value)
        return coef_value, mse_value
    return coef_value

In [8]:
def evaluating_process(img_arr, pixel_size, wave_length, delta, epsilon, crop_size, did_evaluate_mse=False):
    multi_focus_arr = propagating_process(img_arr, pixel_size, wave_length, delta, crop_size)
    coef = []
    if did_evaluate_mse:
        mse = []
    for i in range(len(multi_focus_arr)):
        if did_evaluate_mse:
            coef_value, mse_value = single_evaluate(multi_focus_arr[i], pixel_size,
                                                    wave_length, delta, epsilon, crop_size, did_evaluate_mse=True)
            coef.append(coef_value)
            mse.append(mse_value)
        coef_value = single_evaluate(multi_focus_arr[i], pixel_size,
                                     wave_length, delta, epsilon, crop_size, did_evaluate_mse=False)
        coef.append(coef_value)
    coef_mean = np.mean(coef)
    coef.sort()
    print(coef[:10])
    print("mean of Coef", coef_mean)
    if did_evaluate_mse:
        mse_mean = np.mean(mse)
        print("mean of MSE:", mse_mean)

In [ ]:
img_arr = import_img(data_dir, save_dir, 1000, 128)
evaluating_process(img_arr, 14.8e-6, 632.8e-9, 0.0005, 0, 128)

In [45]:
def whole_processing(img_num, pixel_size=7.4e-6, wave_length=632.8e-9, delta=0.0005, check=False):
    ck = Check()
    file_list = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    crop_box = (100, 25, 500, 425)
#     coef = []
    if img_num <= 5000:
        imgs_arr = np.zeros((img_num, 128, 128, 4))
    else:
        imgs_arr = np.zeros((5000, 128, 128, 4))
    for i in range(img_num):
        amp_dir = file_list[i]
        pha_dir = file_list[10000-i]
        amp_img = import_one_pic(amp_dir, crop_box, 128)
        pha_img = import_one_pic(pha_dir, crop_box, 128)
#         ck.multi_img(amp=amp_img, pha=pha_img)
        amp_img = img_scale(amp_img, img_min=1.0, img_max=1.08)
        pha_img = img_scale(pha_img, img_min=0.0, img_max=1.5)
        wf = WavefrontPropagate(128, pixel_size, wave_length)
        wf.get_obj_wavefront(amp_img, pha_img)
        multi_focus_img = wf.directly_propagate(delta, 128)
        if check:
            ck.directly_propagate(multi_focus_img)
            pxy = tie_algorithm(multi_focus_img[0], multi_focus_img[1], multi_focus_img[2],
                                128, delta, pixel_size, wave_length, 0)
            ground_truth = pha_img
            diff = ground_truth - np.min(ground_truth) - (pxy - np.min(pxy))
            coef_value = pearson_correlation_coefficient(ground_truth, pxy)
            ck.multi_img(False, ground_truth=ground_truth, py_recover=pxy, difference=diff)
            print(i, "  Coef: ", coef_value)
            coef.append(coef_value)
        idx = i % 5000
        imgs_arr[idx, :, :, 0] = multi_focus_img[0]
        imgs_arr[idx, :, :, 1] = multi_focus_img[1]
        imgs_arr[idx, :, :, 2] = multi_focus_img[2]
        imgs_arr[idx, :, :, 3] = pha_img
        if (i+1)%5000 == 0:
            print("Saving...")
            with open(save_dir+"imgs_arr_"+str(img_num)+"_"+str(i//5000), "wb") as wf:
                pickle.dump(imgs_arr, wf)
            print("Saved")
            imgs_arr = np.zeros((5000, 128, 128, 4))
        if i % 10 == 0:
            print(i//5000, ": ", idx)

In [46]:
whole_processing(10000)

0 :  0
0 :  10
0 :  20
0 :  30
0 :  40
0 :  50
0 :  60
0 :  70
0 :  80
0 :  90
0 :  100
0 :  110
0 :  120
0 :  130
0 :  140
0 :  150
0 :  160
0 :  170
0 :  180
0 :  190
0 :  200
0 :  210
0 :  220
0 :  230
0 :  240
0 :  250
0 :  260
0 :  270
0 :  280
0 :  290
0 :  300
0 :  310
0 :  320
0 :  330
0 :  340
0 :  350
0 :  360
0 :  370
0 :  380
0 :  390
0 :  400
0 :  410
0 :  420
0 :  430
0 :  440
0 :  450
0 :  460
0 :  470
0 :  480
0 :  490
0 :  500
0 :  510
0 :  520
0 :  530
0 :  540
0 :  550
0 :  560
0 :  570
0 :  580
0 :  590
0 :  600
0 :  610
0 :  620
0 :  630
0 :  640
0 :  650
0 :  660
0 :  670
0 :  680
0 :  690
0 :  700
0 :  710
0 :  720
0 :  730
0 :  740
0 :  750
0 :  760
0 :  770
0 :  780
0 :  790
0 :  800
0 :  810
0 :  820
0 :  830
0 :  840
0 :  850
0 :  860
0 :  870
0 :  880
0 :  890
0 :  900
0 :  910
0 :  920
0 :  930
0 :  940
0 :  950
0 :  960
0 :  970
0 :  980
0 :  990
0 :  1000
0 :  1010
0 :  1020
0 :  1030
0 :  1040
0 :  1050
0 :  1060
0 :  1070
0 :  1080
0 :  1090
0 :  1100
0

1 :  3400
1 :  3410
1 :  3420
1 :  3430
1 :  3440
1 :  3450
1 :  3460
1 :  3470
1 :  3480
1 :  3490
1 :  3500
1 :  3510
1 :  3520
1 :  3530
1 :  3540
1 :  3550
1 :  3560
1 :  3570
1 :  3580
1 :  3590
1 :  3600
1 :  3610
1 :  3620
1 :  3630
1 :  3640
1 :  3650
1 :  3660
1 :  3670
1 :  3680
1 :  3690
1 :  3700
1 :  3710
1 :  3720
1 :  3730
1 :  3740
1 :  3750
1 :  3760
1 :  3770
1 :  3780
1 :  3790
1 :  3800
1 :  3810
1 :  3820
1 :  3830
1 :  3840
1 :  3850
1 :  3860
1 :  3870
1 :  3880
1 :  3890
1 :  3900
1 :  3910
1 :  3920
1 :  3930
1 :  3940
1 :  3950
1 :  3960
1 :  3970
1 :  3980
1 :  3990
1 :  4000
1 :  4010
1 :  4020
1 :  4030
1 :  4040
1 :  4050
1 :  4060
1 :  4070
1 :  4080
1 :  4090
1 :  4100
1 :  4110
1 :  4120
1 :  4130
1 :  4140
1 :  4150
1 :  4160
1 :  4170
1 :  4180
1 :  4190
1 :  4200
1 :  4210
1 :  4220
1 :  4230
1 :  4240
1 :  4250
1 :  4260
1 :  4270
1 :  4280
1 :  4290
1 :  4300
1 :  4310
1 :  4320
1 :  4330
1 :  4340
1 :  4350
1 :  4360
1 :  4370
1 :  4380
1 :  4390


In [41]:
# img_num = 200
# whole_processing(img_num)
# ck = Check()
# coef = []
# with open(save_dir+"imgs_arr_"+str(img_num), "rb") as rf:
#     imgs_arr = pickle.load(rf)
# for i in range(img_num):
#     ground_truth = imgs_arr[i, :, :, 3]
#     p_xy = tie_algorithm(imgs_arr[i, :, :, 0], imgs_arr[i, :, :, 1], imgs_arr[i, :, :, 2],
#                         128, 0.0005, 7.4e-6, 632.8e-9, 0)
#     diff = ground_truth - np.min(ground_truth) - (p_xy - np.min(p_xy))
#     coef_value = pearson_correlation_coefficient(ground_truth, p_xy)
# #     ck.multi_img(gt=ground_truth, p_xy=p_xy, diff=diff)
#     print(coef_value)
#     coef.append(coef_value)
# print("mean: ", np.mean(coef))

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
Saving...
Saved
0.9875772206877571
0.9609302025226324
0.9382377430368655
0.9434752136982608
0.9773202121717699
0.9596556142331636
0.9774075945296112
0.9396672985088398
0.9481419943471924
0.9602084671222099
0.9786167786594264
0.9773012393164262
0.9339238198009447
0.9481869414206017
0.9830861892019231
0.9900268422226783
0.9488432283135401
0.9480299874227719
0.9427252187726446
0.9439585909208563
0.9353989233508218
0.9505458766077237
0.9497766513705297
0.9568577124669766
0.9353452096724402
0.9514633925795619
0.9484877244636281
0.9447318539056055
0.9629060919265918
0.9746876121246797
0.9562750288223584
0.9707678598912666
0.9411184236815381
0.9396742839313307
0.9877106699523142
0.983320601388586
0.9539457200084281
0.9465029002587222
0.9766200852808898
0.9661364561707279
0.9493928193410185
0.9716568307443094
0.9842809707359906
0.9543992512674694
0.9602687123644186
0.9480762724190739
0.9543422824004544
0.9330428338385349
0.98

In [44]:
5000%5000

0